In [ ]:
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc
import datetime
import pandas as pd
import pickle
import copy


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(first_name, sname, middle_name, birth_date):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    var = frame.loc[(frame['Surname'] == sname) &
                    (frame['Name'] == first_name) &
                    (frame['Patronym'] == middle_name) &
                    (frame['Birth'] == birth_date)]
    exist = var.size > 0
    if exist:
        return sname + " " + first_name + " " + middle_name + " " + birth_date + ": " + "bad_boy"
    else:
        return sname + " " + first_name + " " + middle_name + " " + birth_date + ": " + "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Бинарная передача данных
def send_back_binary(bin_data):
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width):
            if type(img_arr[i][j]) is list:
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
            else:
                img_arr[i][j] = 255 - img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

def binary_val(value, border): 
    if (value < border):
        return 0
    else:
        return 255

def bin_img(bin_data, border1, border2, border3):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width):
            if len(img_arr.shape) > 2:
                img_arr[i][j][0] = binary_val(img_arr[i][j][0], border1)
                img_arr[i][j][1] = binary_val(img_arr[i][j][1], border2)
                img_arr[i][j][2] = binary_val(img_arr[i][j][2], border3)
            else:
                img_arr[i][j] = binary_val(img_arr[i][j], border1)
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(bin_img, 'bin_img')

def reverse(bin_data):
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    for i in range(height):
        for j in range(width // 2):
            (img_arr[i][j], img_arr[i][width - j - 1]) = (copy.copy(img_arr[i][width - j - 1]), copy.copy(img_arr[i][j])) 
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(reverse, 'reverse')
            
            

print("Listening on port 8008...")
server.serve_forever()

Listening on port 8008...


127.0.0.1 - - [24/Oct/2023 18:57:38] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 18:57:40] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 18:57:41] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 18:57:41] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 18:57:41] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 18:57:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 18:57:44] "POST /RPC2 HTTP/1.1" 200 -
